# META PROGRAMAÇÃO

A Metaprogramação  é técnica de criar metaprogramas, ou seja, metaprogramas pode ser definido como um código que criam ou manipulam outros códigos. Bons exemplos de metaprograas são compiladores ou interpretadores. Um bom exemplo do que pode ser feito com metaprogramação é que é possível escrever um código que converte expressões simples em expressões mais complicadas ou examinar um código antes de executá-lo e dessa forma é possível alterá-lo para que ele seja executado mais rapidamente. De acordo com o manual oficial da linguagem Julia, o legado mais forte do Lisp em Julia é o seu apoio a metaprogramação. MATLAB (avaliado até a versão 2016) não oferece nativamente ferramentas  metaprogramação mas com um pouco de criatividade isso pode ser feito usando a `ToolBox` simbólica.

## SIMBOLOS

## MACROS

Macros são pequenos programas que executam uma sequencia de comandos definidos. Os macros são semelhantes as funções, mas em vez de trabalhar com valores, elas trabalham com expressões como argumentos de entrada. Uma macro mapeia uma tupla de argumentos para uma expressão retornada e a expressão resultante é compilada diretamente. Os argumentos de um macro podem incluir expressões, valores literais e símbolos. Resumindo:
- `Função`: trabalha os valores de entrada e retorna os valores calculados em tempo de execução
- `Macro`: Faz a análise das expressões da entrada e retorna as expressões modificadas em tempo de análise.

Macros são necessários porque eles executam quando o código é analisado, portanto, macros permitem que o programador possa gerar e incluir fragmentos de código personalizado antes que o programa completo seja executado. Se um macro é usada dentro de uma função, ele é executada quando a função é definida, antes que a função seja compilada ou executada. 

#### Macros Importantes

**@time, @timev, @timed e @elapsed **



**@evalpoly**

Calcula o polinômio ${\displaystyle p(x) = a_{0}+a_{1}x^1+a_{2}x^{2}+a_{3}x^{3}+\cdots +a_{n}x^{n},}$
. Essa macro se expande para um código inline eficiente que usa o método de Horner ou, para o z complexo, um algoritmo de Goertzel mais eficiente. Sintaxe:
```julia
@evalpoly k a_0 a_1 a_2 ... a_n
```
Sendo: 
* **k**: constante;
* **a_0, a_1, a_2, a_n**: os coeficientes do polinômio

Exemplo: calcular o polinômio $P= 1 + 2x + 3x^2 + 4x^3$ para $x = 0$

In [57]:
@evalpoly 0 1 2 3 4

1

**@which**

Aplicado a uma chamada de função ou macro, ele avalia os argumentos para a chamada especificada e retorna o objeto Method para o método que seria chamado para esses argumentos. Aplicado a uma variável, retorna o módulo no qual a variável foi vinculada. Ele chama para a função que.

In [4]:
f(x::Float64, y::Float64) = x^2 + y^2
f(x::Int32, y::Int32) = x^2 + y^2

f (generic function with 2 methods)

In [25]:
@which f(2.8, 5.2)

f(x::Float64, y::Float64) in Main at In[4]:1

\check


\check

** Método de Horner** 

O Algoritmo de Horner, dedicado ao matemático William George Horner, é um algoritmo para calcular de forma eficiente funções polinomiais de uma forma mais simples. Dado o polinômio:

${\displaystyle p(x)=a_{0}+a_{1}x+a_{2}x^{2}+a_{3}x^{3}+\cdots +a_{n}x^{n},}$

No qual $ a_{0},\ldots ,a_{n}$ são números reais, o polinômio é calculado para um valor inicial $x_0$. Para realizar os procedimentos seguintes, definimos uma nova sequência de constantes como mostrado abaixo:

$b_n = a_n$

$b_{n-1} = a_{n-1} + b_n x$

$b_{n-2} = a_{n-2} + b_{n-1} x$

$\vdots$

$b_{0} = a_{0} + b_a x$

Logo $b_0$ é o valor do polinômio $p(x_0)$

Calcular o polinômio $10 + 1x + 2x^2 + 3x^3 + 4x^4
$

In [26]:
function pol_horner(x, a...)
    b = 0.0
    println(a)
    for i = length(a): -1: 1
        b = a[i] + b*x
    end
    return b
end

pol_horner (generic function with 1 method)

In [27]:
pol_horner(0, 10, 1, 2, 3, 4)

(10, 1, 2, 3, 4)


10.0

In [28]:
using BenchmarkTools

ArgumentError: ArgumentError: Package BenchmarkTools not found in current path:
- Run `import Pkg; Pkg.add("BenchmarkTools")` to install the BenchmarkTools package.


In [28]:
@elapsed pol_horner(3.5, 10, 1, 2, 3, 4)

(10, 1, 2, 3, 4)


0.009630368

In [29]:
@elapsed @evalpoly 3.5 10 1 2 3 4

0.016751035

In [37]:
# para 10 + 5.1x - 2.0x^2 - 0.5x^3 + x^5 + 1.1x^7
b = rand(10000);
for i = 1:10000
    b[i] = @elapsed @evalpoly(1.25,10, 5.1, 2, 0, -0.5, 0, 1, 0, rand()  );
end
b1 = float(minimum(b))

2.5e-8

In [31]:
BigFloat(a1), BigFloat(b1)

UndefVarError: UndefVarError: a1 not defined

In [32]:
macro macro_horner(x, p...)
    ex = esc(p[end])
    
    for i = length(p)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(p[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@macro_horner (macro with 1 method)

In [33]:
@timev @macro_horner(3.5, 10, 1, 2, 3, 4)

  0.000005 seconds (9 allocations: 272 bytes)
elapsed time (ns): 4731
bytes allocated:   272
pool allocs:       9


766.875

In [36]:
# para 10 + 5.1x - 2.0x^2 - 0.5x^3 + x^5 + 1.1x^7
a = rand(10000)
for i = 1:10000
    a[i] =  @elapsed @macro_horner(1.25,10, 5.1, 2, 0, -0.5, 0, 1, 0, randn())
end
minimum(a)

2.3e-8

In [131]:
ex = esc((1,2,3)[end])

:($(Expr(:escape, 3)))

## BIBLIOGRAFIA

* Sebesta, Robert W.Conceitos de linguagens de programação [recurso eletrônico] /Robert W. Sebesta; tradução técnica: Eduardo Kessler Piveta. – 9. ed. – Dados eletrônicos. – Porto Alegre : Bookman, 2011.

* Vieira, Francisco. Programação Funcional usando Haskell. Teresina, 2005. 

